# Debiasing word embeddings

Word embedding are word vectors that have meaning, word vectors similar to each other will be close to each other in a vector space. 

**After completing this lab you will be to:**

- Measure similarity of word vectors using cosine similarity
- Solve word analogy probelms such as Man is to Woman as Boy is to ____ using word embeddings
- Reduce gender bias in word embeddings by modifying word embeddings to remove gender stereotypes, such as the association between the words *receptionist* and *female*

## <font color='darkblue'>Word embeddings</font>

Word embedding is a method used to represent words as vectors. They are populary used in machine learning and natural language processing tasks. Despite their success in downstream tasks such as cyberbullying, sentiment analysis, and question retrieval, they exhibit gender sterotypes which raises concerns because their widespread use can amplify these biases. 


Word embeddings are trained using a machine learning algorithm on word co-occurance using a text dataset. After training , each word will be represented as a vector. **You can think of a vector as a list of numbers. When convenient we will refer to a vector of a word using an arrow on top of the word. But in most part of the lab the arrow will be omitted for simplicity.**

#### Word embedding properties:
* Words with similar semantic meaning will be close to each other 
* The difference between word embedding vectors can represent relationships between words. For example, given the analogy "man is to King as woman is to $x$" (denoted as $man:king :: woman:x$), by doing simple arithmetic on the embedding vectors, we find that $x = queen$ is the best answer because $\vec{man} - \vec{woman} ≈ \vec{king} - \vec{queen}$. For the analogy $Paris:France :: Nairobi:x$, finds that $x = Kenya$. These embeddings can also amplify sexism implicit in text. For instance, $\vec{man} - \vec{woman} ≈ \vec{computer \space programmer} - \vec{homemaker}$. The same system that produced reasonable answers to the previous examples offensively answers "man is to computer programmer as woman is to $x$" with $x = homemaker$.

Run the following cell to load the required modules. 

In [ ]:
#@title
import os
import json
import numpy as np
from pathlib import Path
from sklearn.decomposition import PCA

## <font color='darkblue'>Download and Load word vectors</font>
Due to the computational resources required to train word embeddings, we will be using a pre-trained word embeddings known as GloVe. 

Run the following cells to download and load the word embeddings.

In [ ]:
#@title
def download_glove_vectors():
    '''
    Download the GloVe vectors
    Arguments:
        None
    Returns:
        file_name (String): The absolute path of the downloaded 50-dimensional
        GloVe word vector representations
    '''
    
    if not Path('data').is_dir():
        print("Downloading the word embeddings ...")
        !wget --quiet https://nlp.stanford.edu/data/glove.6B.zip
        print("Word embeddings downloaded.")
    
        # Unzip it
        print("Unzipping the downloaded file ...")
        !unzip -q glove.6B.zip -d data/ 
        print("File unzipped.")

    return '/content/data/glove.6B.50d.txt'

In [ ]:
#@title
def get_glove_vectors(glove_file):
    '''
    Read the word vectors in glove_file
    Arguments:
        glove_file (String): The absolute path to the downloaded glove word embeddings
    Returns:
        words (Set): The words (vocabulary) in the pretrained glove word embeddings
        word_to_vector_map (Dict): A dictionary mapping the each word to its embedding vector 
    '''

    words = set()
    word_to_vector_map = {}
    with open(glove_file, 'r') as file_handle:
        for line in file_handle:
            line = line.strip().split()
            current_word = line[0]
            words.add(current_word)
            current_word_vector = line[1:]
            word_to_vector_map[current_word] = np.array(current_word_vector, dtype=np.float64)

    return words, word_to_vector_map

Download and load words as vectors(a list of numbers)

In [ ]:
#@title
# Load sets of words in the vocabulary and a dictionary mapping words to their GloVe vectors 
words, word_to_vector_map = get_glove_vectors(download_glove_vectors())

How many words are in our vocabulary

In [ ]:
#@title
print(f"Vocabulary contains {len(word_to_vector_map)} words")

Vocabulary contains 400000 words


View a few words by running the cell below

In [ ]:
#@title
print(f'Doctor is represented as: \n {word_to_vector_map["doctor"]}')
print()
print(f'School is represented as: \n {word_to_vector_map["school"]}')

Try your own output. Run the cell below and enter a word then **hit enter or return** on your keyword.

In [ ]:
#@title
import ipywidgets as widgets
from IPython.display import display

text = widgets.Text(description='Enter a word:', value='', disabled=False)
display(text)

def display_input_text_vector(input):
    try:
        vector = word_to_vector_map[input.value]
    except KeyError:
        print(f'{input.value} is not in our vocabulary. Give another word a shot.')
        return
    print(f'{input.value} is represented as: \n{vector}')

text.on_submit(display_input_text_vector)


## <font color='darkblue'>Operations on word embeddings</font>

### Task 1 - Cosine similarity

Similarity between two words represented as words $wordA$ and $wordB$ can be measured by their cosine similarity:

$$\text{CosineSimilarity(wordA, wordB)} = \frac {wordA \cdot wordB} {||wordA||_2 ||wordB||_2} \tag{1}$$

The above equation returns a value that denotes the level of similarity between $wordA$ and $wordB$. 

The cosine similarity of $wordA$ and $wordB$ will be close to 1 if the two words are similar, otherwise, the cosine similarity will be small. 

**Note**: We will be using "vector or embedding of a word" and "word" interchangeably in this lab. 

Run the code cell below to execute the formula above. 



In [ ]:
#@title
def cosine_similarity(vector1, vector2):
    """
    Calculates the cosine similarity of two word vectors - vector1 and vector2
    Arguments:
        vector1 (ndarray): A word vector having shape (n,)
        vector2 (ndarray): A word vector having shape (n,)
    Returns:
        cosine_similarity (float): The cosine similarity between vector1 and vector2
    """

    # Compute the dot product between vector1 and vector2 
    dot = np.dot(vector1, vector2)                                             

    # Compute the Euclidean norm or length of vector1 
    norm_vector1 = np.sqrt(np.sum(vector1 ** 2))  

    # Compute the Euclidean norm or length of vector2 
    norm_vector2 = np.sqrt(np.sum(vector2 ** 2)) 

    # Compute the cosine similarity as defined in equation 1 
    cosine_similarity = dot / (norm_vector1 * norm_vector2)                    

    return cosine_similarity


***
**<font color='red'>Task 1:</font>** 

1) Which of the outputs are very similar or dissimilar? Give a reason for each of your answer. 

2) What can you observe about the last output? 
***

In [ ]:
#@title
# Run this cell to obtain and report your answers
man = word_to_vector_map["man"]
woman = word_to_vector_map["woman"]
cat = word_to_vector_map["cat"]
dog = word_to_vector_map["dog"]
orange = word_to_vector_map["orange"]
england = word_to_vector_map["england"]
london = word_to_vector_map["london"]
edinburgh = word_to_vector_map["edinburgh"]
scotland = word_to_vector_map["scotland"]

print(f"Cosine similarity between man and woman: {cosine_similarity(man, woman)}")
print(f"Cosine similarity between cat and dog: {cosine_similarity(cat, dog)}")
print(f"Cosine similarity between cat and cow: {cosine_similarity(cat, orange)}")
print(f"Cosine similarity between england - london and edinburgh - scotland: {cosine_similarity(england - london, edinburgh - scotland)}")

### Task 2 - Word analogy

In an analogy task, you are given an analogy in the form "i is to j as k is to ___". Your task is to complete this sentence. 

For example, if you are given "man is to king as woman is to $l$" (denoted as $man:king :: woman:l$). You are to find the best word $l$ that answers the analogy the best. Simple arithmetic of the word embeddings will find that $l = queen$ is the best answer because the embedding vectors of words $i$, $j$, $k$, and $l$ denoted as $e_i$, $e_j$, $e_k$, $e_l$ have the following relationship:
$$e_j - e_i ≈ e_l - e_k$$

Cosine similarity can be used to measure the similarity between $e_j - e_i$ and $e_l - e_k$

Run the cell below to setup the analogy task.

In [ ]:
#@title
def answer_analogy(word_i, word_j, word_k, word_to_vector_map):
    """
    Performs word analogy as described above
    Arguments:
        word_i (String): A word
        word_j (String): A word
        word_k (String): A word
        word_to_vector_map (Dict): A dictionary of words as key and its associated embedding vector as value
    Returns:
        best_word (String): A word that fufils the relationship that e_j - e_i as close as possible to e_l - e_k, as measured by cosine similarity
    """

    # Convert words to lowercase
    word_i = word_i.lower()
    word_j = word_j.lower()
    word_k = word_k.lower()

    # Start code here #
    try:
        # Get the embedding vectors of word_i (~ 1 line)
        embedding_vector_of_word_i = word_to_vector_map[word_i]          # Replace with None
    except KeyError:
        print(f"{word_i} is not in our vocabulary. Please try a different word.")
        return

    try:
        # Get the embedding vectors of word_j (~ 1 line)
        embedding_vector_of_word_j = word_to_vector_map[word_j]          # Replace with None
    except KeyError:
        print(f"{word_j} is not in our vocabulary. Please try a different word.")
        return

    try:
        # Get the embedding vectors of word_k (~ 1 line)
        embedding_vector_of_word_k = word_to_vector_map[word_k]          # Replace with None
    except KeyError:
        print(f"{word_k} is not in our vocabulary. Please try a different word.")
        return
    # End code here #

    # Get all the words in our word to vector map i.e our vocabulary
    words = word_to_vector_map.keys()
    max_cosine_similarity = -1000                           # Initialize to a large negative number
    best_word = None                                        # Keep track of the word that best answers the analogy. Note: Do not change this None

    # Since we are looping through the whole vocabulary, if we encounter a word 
    # that is the same as our input, that word becomes the best_word. To avoid 
    # that we skip the input word. 
    input_words = set([word_i, word_j, word_k])

    for word in words:
        if word in input_words:
            continue
        
        # Start code here #
        # Compute cosine similarity 
        similarity = cosine_similarity(
            embedding_vector_of_word_j - embedding_vector_of_word_i,
            word_to_vector_map[word] - embedding_vector_of_word_k
        )                                                # Replace arguments with None

        # Have we seen a cosine similarity bigger than max_cosine_similarity?
            # then update the max_cosine_similarity to the current cosine similarity
            # and update the best_word to the current word (~ 3 lines) 
        if similarity > max_cosine_similarity:           # Replace with None > None
            max_cosine_similarity = similarity           # Replace with max_cosine_similarity = None
            best_word = word                             # Replace with best_word = None
        # End code here

    return best_word

***
**<font color='red'>Task 2:</font>** After running the cell below;


1) What do you observe? Were there any cultural or gender stereotypes? List them. 

2) Does the algorithm always give the right answer? List the incorrect analogies and what the correct analogy is if any. 
***


In [ ]:
#@title
analogies = [('france', 'french', 'germany'), 
             ('england', 'london', 'japan'), 
             ('boy', 'girl', 'man'), 
             ('man', 'doctor', 'woman'),
             ('small', 'smaller', 'big')]
for analogy in analogies:
    best_word = answer_analogy(*analogy, word_to_vector_map)
    if best_word:
        print(f"{analogy[0]} -> {analogy[1]} :: {analogy[2]} -> {best_word}")

### Task 3 - Geometry of Gender and Bias in Word Embeddings: Occupational stereotypes 
In this task, we will understand the biases present in word-embedding i.e which words are closer to $she$ than to $he$. This will be achieved by evaluating whether the pre-trained word embeddings have sterotypes on occupation words. 

We can determine gender bias by computing the cosine similarity between each occupation word embedding and the embedding vector obtained by subtracting the embedding vector of $he$ from that of $she$ i.e ($she - he$).

$$occupation\_word_i \cdot ||she - he||_2 \tag{2}$$

Run the cells below to download and view the occupations.

In [ ]:
#@title
def download_occupations():
    if not Path('debiaswe').is_dir():
        print("Downloading occupation list ...")
        !git clone -q https://github.com/tolga-b/debiaswe.git
        print("Occupation list downloaded.")

    return '/content/debiaswe/data/professions.json'


def view_occupations(occupations_file):
    with open(occupations_file, 'r') as file_handle:
        occupations = json.load(file_handle)

        for occupation in occupations:
            print(occupation[0])

Download occupations list

In [ ]:
#@title
occupations_file = download_occupations()

View list of occupations

In [ ]:
#@title
# View occupations
view_occupations(occupations_file)

Execute the cell below to setup similarity computation between the word embedding vector of occupation words and the embedding vector difference between $she$ and $he$.

In [ ]:
#@title
def get_occupation_stereotypes(she, he, occupations_file, word_to_vector_map, verbose=False):
    """
    Computes the words that are closest to she and he in the GloVe embeddings
    Arguments:
        she (String): A word
        he (String): A word
        occupations_file (String): The path to the occupation file
        word_to_vector_map (Dict): A dictionary mapping words to embedding vectors
    Returns:
        most_similar_words (Tuple(List[Tuple(Float, String)], List[Tuple(Float, String)])): 
        A tuple of the list of the most similar occupation words to she and he with their associated similarity
    """

    # Read occupations
    with open(occupations_file, 'r') as file_handle:
        occupations = json.load(file_handle)

    # Extract occupation words
    occupation_words = [occupation[0] for occupation in occupations]

    # Get embedding vector of she 
    embedding_vector_she = word_to_vector_map[she]                                                        
    # Get embedding vector of he                                                               
    embedding_vector_he = word_to_vector_map[he]                                                         
    # Get the vector difference between embedding vectors of she and he          
    vector_difference_she_he = embedding_vector_she - embedding_vector_he                                
    # Get the normalized difference 
    normalized_difference_she_he = vector_difference_she_he / np.linalg.norm(vector_difference_she_he)   

    # Store the cosine similarities
    similarities = []

    for word in occupation_words:
        try:
            # Get the embedding vector of the current occupation word
            occupation_word_embedding_vector = word_to_vector_map[word]                                                                  
            # Compute cosine similarity between embedding vector of the occupation word and normalized she - he vector 
            similarity = np.dot(occupation_word_embedding_vector, normalized_difference_she_he)                                           
            similarities.append((similarity, word))
        except KeyError:
            if verbose:
                print(f"{word} is not in our vocabulary.")

    most_similar_words = sorted(similarities)

    return most_similar_words[:20], most_similar_words[-20:]



***
**<font color='red'>Task 3:</font>** Execute the cell below and report your results. 

1) Does the GloVe word embeddings propagate bias? why?

2) From the list associated with she, list those that reflect gender stereotype.   

3) Compare your list from 2 to the occupations closest to he. What are your conclusions?

Exclude businesswoman from your list.
***

In [ ]:
#@title
he, she = get_occupation_stereotypes('she', 'he', occupations_file, word_to_vector_map)
print("Occupations closest to he:")
for occupation in he:
    print(f"{occupation[0], occupation[1]}")

print("\nOccupations closest to she:")
for occupation in she:
    print(f"{occupation[0], occupation[1]}")

<!--
### Task 4 - Analogies exhibiting stereotypes 
Using analogies to quantify gender stereotype in the embedding. Given two words, e.g. $he$, $she$, generate a pair of words, $x$ and $y$, such that $he$ to $x$ as $she$ to $y$ is a good analogy. This will generate pairs that the embedding believes to be analogous to $he$, $she$ or any other pair of seed words. 

The analogy generator takes as input a pair of seed words (a, b) which determines the seed direction $\vec{a} - \vec{b}$ corresponding to the ***normalized*** difference between the two seed words. 

In this task, we will use $(a, b) = (she, he)$. Then all pairs of words $x, y$ is scored using the following metric:

$$S_{(a, b)}(x, y) = ||\vec{a} - \vec{b}||_2 \cdot ||\vec{x} - \vec{y}||_2 \space \text{if} \space ||\vec{x} - \vec{y}||_2 ≤ δ, 0 \space \text{else}$$

Where $δ$ is a threshold for semantic similarity. We will use $δ = 1$. 

<!-- In other words, the above equation reads, if the normalized difference between $x$ and $y$ is less than or equal to our threshold, then the score of the pair of words $x, y$ is the dot product of the normalized difference between the seed pairs and the normalized difference between the pair of words $x, y$. ->

Notice that each vector difference is normalized, therefore we are basically computing the numerator of equation 1 as part of this equation.

***
**<font color='red'>Task 4:</font>** Test the implementation of your `get_analogies_exhibiting_stereotypes()` below. Report your results. Are the generated analogies biased?
***
-->

### Task 4 - Debiasing word embeddings 

**Gender Specific words**

Words that are associated with a gender by definition. For example, brother, sister, businesswoman or businessman. 

**Gender neutral words**

The remanining words that are not specific to a gender are gender neutral. For example, flight attendant or shoes. The compliment of gender specific words, can be taken as the gender neutral words. 

**Step 1 - Identify gender subspace i.e the embedding vector (list of numbers) that captures the bias**

To robustly estimate bias, we use the gender specific words to learn a gender subpace. To identify the gender subspace, we consider the vector difference of gender specific word pairs, such as $\vec{she} - \vec{he}$, $\vec{woman} - \vec{man}$ or $\vec{her} - \vec{his}$. This identifies a **gender embedding that captures gender**. Aggregating over the word vector differences gives a more accurate embedding vector that captures gender bias. 

Run the cell below to setup the computation that returns the embedding that captures gender bias.


In [ ]:
#@title
from sklearn.decomposition import PCA

def get_gender_subspace(pairs, word_to_vector_map, num_components=10):
    """
    Compute the gender subspace by computing the principal components of 
    ten gender pair vectors. 
    Arguments:
        pairs (List[Tuple(String, String)]): A list of gender specific word pairs
        word_to_vector_map (Dict): A dictionary mapping words to embedding vectors
        num_components (Int): The number of principal components to compute. Defaults to 10
    Returns:
        gender_subspace (ndarray): The gender bias subspace(or direction) of shape (embedding dimension,)
    """

    matrix = []
    for word_1, word_2 in pairs:
        embedding_vector_word_1 = word_to_vector_map[word_1]
        embedding_vector_word_2 = word_to_vector_map[word_2]
        center = (embedding_vector_word_1 + embedding_vector_word_2) / 2
        matrix.append(embedding_vector_word_1 - center)
        matrix.append(embedding_vector_word_2 - center)
    
    matrix = np.array(matrix)
    pca = PCA(n_components=num_components)
    pca.fit(matrix)

    pcs = pca.components_                  # Sorted by decreasing explained variance
    eigenvalues = pca.explained_variance_  # Eigenvalues 
    gender_subspace = pcs[0]               # The first element has the highest eigenvalue
    return gender_subspace

Run the following cell to view the gender embedding 

In [ ]:
#@title
gender_specific_pairs = [
    ('she', 'he'),
    ('her', 'his'),
    ('woman', 'man'),
    ('mary', 'john'),
    ('herself', 'himself'),
    ('daughter', 'son'),
    ('mother', 'father'),
    ('gal', 'guy'),
    ('girl', 'boy'),
    ('female', 'male')
]
gender_direction = get_gender_subspace(gender_specific_pairs, word_to_vector_map)
print(f'Gender embedding is: \n{gender_direction}')

 
***
**<font color='red'>Task 4a:</font>** Running the cell below computes the similarity between the gender embedding and the embedding vectors of male and female names. What can you observe about the names? 
***

In [ ]:
#@title
print('Names and their similarities')
names = ["mary", "john", "sweta", "david", "kazim", "angela"]
for name in names:
    print(name, cosine_similarity(gender_direction, word_to_vector_map[name]))

***
**<font color='red'>Task 4b:</font>** Quantify direct and indirect biases between words and the gender embedding by running the following cell. What is your observation?
***

In [ ]:
#@title
words = ["engineer", "science", "pilot", "technology", "lipstick", "arts", "singer", "computer", "receptionist", "fashion", "doctor", "literature"]
for word in words:
    print(word, cosine_similarity(gender_direction, word_to_vector_map[word]))

**Step 2 - Neutralize gender neutral words**

Words such as "babysit" should remain neutral in association with gender specific words. "babysit" shouldn't be more closer to "grandmother" than to "grandfather". Take college admission as an example, we do not want the embedding of a candidate to be prefered over another by gender. In such cases it would be a good idea to neutralize the candidates or "babysit" by ensuring the numbers in their embedding list is zero in the gender subspace. 

Run the cell below to setup neutralization. 

In [ ]:
#@title
def neutralize(word, gender_direction, word_to_vector_map):
    """
    Project the vector of word onto the gender subspace to remove the bias of "word"
    Arguments:
        word (String): A word to debias
        gender_direction (ndarray): Numpy array of shape (embedding size (50), ) which is the bias axis
        word_to_vector_map (Dict): A dictionary mapping words to embedding vectors

    Returns:
        debiased_word (ndarray): the vector representation of the neutralized input word 
    """

    # Get the vector representation of word 
    embedding_of_word = word_to_vector_map[word]

    # Compute the projection of word onto gender direction 
    projection_of_word_onto_gender = (np.dot(embedding_of_word, gender_direction) * gender_direction) / np.sum(gender_direction ** 2)

    # Neutralize word
    debiased_word  = embedding_of_word - projection_of_word_onto_gender

    return debiased_word


***
**<font color='red'>Task 4c:</font>** Run the cell below to neutralize the gender neutral word "babysit". What is your observation?
***

In [ ]:
#@title
word = "babysit"
print(f"Before neutralization, cosine similarity between {word} and embedding that captures gender bias is: \n{cosine_similarity(word_to_vector_map[word], gender_direction)}")

debiased_word = neutralize(word, gender_direction, word_to_vector_map)
print(f"After neutralization, cosine similarity between {word} and embedding that captures gender bias  is: \n{cosine_similarity(debiased_word, gender_direction)}")

**Step 3 - Equalize**

Neutralization debiases gender neutral words, however, it does not respect the definition of gender specific words. Equalization fixes this by ensuring that all neutral words are equidistant to all words in pair or set of gender specific words. 

Going back to our "babysit" example, if {grandmother, grandfather} and {guy, gal} are two gender specific sets, then after performing equalization, "babysit" should be of equal distance to $grandmother$ and $granfather$ and equal distance to $guy$ and $gal$. But closer to the grandparents and further from the $guy$ and $gal$. 

This handles the situation where bias doesn't be displayed with respect to neutral words. 

Run the cell below to setup neutralization computation. 

In [ ]:
#@title
def equalization(equality_set, bias_direction, word_to_vector_map):
    """
    Equalize the pair of gender specific words in the equality set ensuring that
    any neutral word is equidistant to all words in the equality set. 
    Arguments:
        equality_set (Tuple(String, String)): a tuple of strings of gender specific
        words to debias e.g ("grandmother", "grandfather")
        bias_direction (ndarray): numpy array of shape (embedding dimension,). The 
        embedding vector representing the bias direction
        word_to_vector_map (Dict):  A dictionary mapping words to embedding vectors
    Returns:
        embedding_word_a (ndarray): numpy array of shape (embedding dimension,). The 
        embedding vector representing the first word
        embedding_word_b (ndarray): numpy array of shape (embedding dimension,). The 
        embedding vector representing the second word
    """

    # Get the vector representation of word pair
    word_a, word_b = equality_set                       
    embedding_word_a = word_to_vector_map[word_a]       
    embedding_word_b = word_to_vector_map[word_b]       

    # Compute the mean (eq. 5) of embedding_word_a and embedding_word_a 
    mean = (embedding_word_a + embedding_word_b) / 2    

    # Compute the projection of mean representation onto the bias direction (eq. 6) 
    mean_B = (np.dot(mean, bias_direction) / np.sum(bias_direction ** 2)) * bias_direction

    # Compute the projection onto the orthogonal subspace (eq. 7) 
    mean_othorgonal = mean - mean_B

    # Compute the projection of th embedding of word a onto the bias direction (eq. 8) 
    embedding_word_a_on_bias_direction = (np.dot(embedding_word_a, bias_direction) / np.sum(bias_direction ** 2)) * bias_direction

    # Compute the projection of th embedding of word b onto the bias direction (eq. 9) 
    embedding_word_b_on_bias_direction = (np.dot(embedding_word_b, bias_direction) / np.sum(bias_direction ** 2)) * bias_direction

    # Re-embed embedding of word a using eq. 10 
    new_embedding_word_a_on_bias_direction = (np.sqrt(np.abs(1 - np.sum(mean_othorgonal ** 2)))) * ((embedding_word_a_on_bias_direction - mean_B) / np.linalg.norm((embedding_word_a - mean_othorgonal) - mean_B))

    # Re-embed embedding of word b using eq. 11 
    new_embedding_word_b_on_bias_direction = (np.sqrt(np.abs(1 - np.sum(mean_othorgonal ** 2)))) * ((embedding_word_b_on_bias_direction - mean_B) / np.linalg.norm((embedding_word_b - mean_othorgonal) - mean_B))

    # Equalize embedding of word a using eq. 12 
    embedding_word_a =  mean_othorgonal + new_embedding_word_a_on_bias_direction

    # Equalize embedding of word b using eq. 13 
    embedding_word_b = mean_othorgonal + new_embedding_word_b_on_bias_direction

    return embedding_word_a, embedding_word_b

Run the cell below perform equalization of the pair (father, mother).

In [ ]:
#@title
print("Cosine similarity before equalization:")
print(f"(embedding vector of father, gender_direction): {cosine_similarity(word_to_vector_map['father'], gender_direction)}")
print(f"(embedding vector of mother, gender_direction): {cosine_similarity(word_to_vector_map['mother'], gender_direction)}")
print()

embedding_word_a, embedding_word_b  = equalization(("father", "mother"), gender_direction, word_to_vector_map)
print("Cosine similarity after equalization:")
print(f"(embedding vector of father, gender_direction): {cosine_similarity(embedding_word_a, gender_direction)}")
print(f"(embedding vector of mother, gender_direction): {cosine_similarity(embedding_word_b, gender_direction)}")

***
**<font color='red'>Task 5a:</font>** Looking at the equalization, what can you observe?.
***

**References**:
 - The debiasing algorithm is from Bolukbasi et al., 2016 [Man is to Computer Programmer as Woman is to Homemake? Debiasing word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)
 - The code is partly adapted from Andrew Ng's debiasing word embeddings course on [Coursera](https://www.coursera.org/learn/nlp-sequence-models/lecture/zHASj/debiasing-word-embeddings)
 - The GloVe word embeddings is publicly available at (https://nlp.stanford.edu/projects/glove/) and is due to the works of Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 